In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [2]:
# Выбор устройства: использование GPU, если доступно, иначе CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Загрузка текста из PDF файла
def load_pdf_text(file_path, start_page=None, end_page=None):
    # Используем PyPDFLoader для загрузки текста из PDF
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    
    # Если указаны начальная и конечная страницы, выбираем только нужные страницы
    if start_page is not None and end_page is not None:
        documents = documents[start_page:end_page]
    
    # Объединяем содержимое страниц в один текст
    text = "".join([doc.page_content for doc in documents])
    return text

# Предобработка текста с использованием CharacterTextSplitter
splitter = CharacterTextSplitter()
def preprocess_text(text):
    documents = splitter.split_text(text)
    return " ".join(documents).lower()

In [4]:
# Загрузка и предобработка текста (замените на путь к вашему файлу)
file_path = 'KGBT+.pdf'
# Можно указать start_page и end_page для загрузки определенного диапазона страниц
text = load_pdf_text(file_path, start_page=6, end_page=25)

incorrect startxref pointer(1)
parsing for Object Streams


In [5]:
text

'В.\xa0 О.\xa0 Пелевин.\xa0 «KGBT+ (КГБТ+)»\n7\xa0\nThe Straight Man. Дом Бахии\n\xa0\nВ наш грозный двадцатый век с его верой в могущество разума и «коллективное творче-\nство масс» (певцы прогресса отчего-то не узнают в нем отката к пирамидам), солдатом быть\nпочетно, а служителем культа стыдно. Достоинства и недостатки моего происхождения, таким\nобразом, уравновешивали друг друга.\nНе буду называть своего прежнего имени. Оно теперь не играет роли (в конце рассказа\nпричина станет ясна). Я отпрыск самурайского рода, отличившегося в войнах, давшего стране\nмного воинов и – нелепое, но обычное соседство – буддийских бонз. По семейной традиции\nя должен был стать священником, а со временем – настоятелем небольшого храма недалеко\nот Токио.\nС раннего детства я мечтал о творчестве. Увы, я был начисто лишен талантов. Кое-чего\nдостиг только в каллиграфии и, как ни странно, кукольном искусстве.\nНедалеко от моего дома жил старый мастер, делавший кукол хина и муса для ежегодного\nкукольног

In [6]:
text = preprocess_text(text)

In [7]:
text

'в.\xa0 о.\xa0 пелевин.\xa0 «kgbt+ (кгбт+)»\n7\xa0\nthe straight man. дом бахии\n\xa0\nв наш грозный двадцатый век с его верой в могущество разума и «коллективное творче-\nство масс» (певцы прогресса отчего-то не узнают в нем отката к пирамидам), солдатом быть\nпочетно, а служителем культа стыдно. достоинства и недостатки моего происхождения, таким\nобразом, уравновешивали друг друга.\nне буду называть своего прежнего имени. оно теперь не играет роли (в конце рассказа\nпричина станет ясна). я отпрыск самурайского рода, отличившегося в войнах, давшего стране\nмного воинов и – нелепое, но обычное соседство – буддийских бонз. по семейной традиции\nя должен был стать священником, а со временем – настоятелем небольшого храма недалеко\nот токио.\nс раннего детства я мечтал о творчестве. увы, я был начисто лишен талантов. кое-чего\nдостиг только в каллиграфии и, как ни странно, кукольном искусстве.\nнедалеко от моего дома жил старый мастер, делавший кукол хина и муса для ежегодного\nкукольног

In [8]:
# Создание отображений символов (создание словаря символов для их преобразования в индексы)
chars = sorted(list(set(text)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

In [9]:
char_to_idx

{'\n': 0,
 ' ': 1,
 '(': 2,
 ')': 3,
 '*': 4,
 '+': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 ';': 20,
 '?': 21,
 'a': 22,
 'b': 23,
 'd': 24,
 'e': 25,
 'g': 26,
 'h': 27,
 'i': 28,
 'j': 29,
 'k': 30,
 'm': 31,
 'n': 32,
 'o': 33,
 'r': 34,
 's': 35,
 't': 36,
 'u': 37,
 'v': 38,
 'w': 39,
 'y': 40,
 '\xa0': 41,
 '«': 42,
 '»': 43,
 'а': 44,
 'б': 45,
 'в': 46,
 'г': 47,
 'д': 48,
 'е': 49,
 'ж': 50,
 'з': 51,
 'и': 52,
 'й': 53,
 'к': 54,
 'л': 55,
 'м': 56,
 'н': 57,
 'о': 58,
 'п': 59,
 'р': 60,
 'с': 61,
 'т': 62,
 'у': 63,
 'ф': 64,
 'х': 65,
 'ц': 66,
 'ч': 67,
 'ш': 68,
 'щ': 69,
 'ъ': 70,
 'ы': 71,
 'ь': 72,
 'э': 73,
 'ю': 74,
 'я': 75,
 '–': 76,
 '…': 77}

In [10]:
idx_to_char

{0: '\n',
 1: ' ',
 2: '(',
 3: ')',
 4: '*',
 5: '+',
 6: ',',
 7: '-',
 8: '.',
 9: '0',
 10: '1',
 11: '2',
 12: '3',
 13: '4',
 14: '5',
 15: '6',
 16: '7',
 17: '8',
 18: '9',
 19: ':',
 20: ';',
 21: '?',
 22: 'a',
 23: 'b',
 24: 'd',
 25: 'e',
 26: 'g',
 27: 'h',
 28: 'i',
 29: 'j',
 30: 'k',
 31: 'm',
 32: 'n',
 33: 'o',
 34: 'r',
 35: 's',
 36: 't',
 37: 'u',
 38: 'v',
 39: 'w',
 40: 'y',
 41: '\xa0',
 42: '«',
 43: '»',
 44: 'а',
 45: 'б',
 46: 'в',
 47: 'г',
 48: 'д',
 49: 'е',
 50: 'ж',
 51: 'з',
 52: 'и',
 53: 'й',
 54: 'к',
 55: 'л',
 56: 'м',
 57: 'н',
 58: 'о',
 59: 'п',
 60: 'р',
 61: 'с',
 62: 'т',
 63: 'у',
 64: 'ф',
 65: 'х',
 66: 'ц',
 67: 'ч',
 68: 'ш',
 69: 'щ',
 70: 'ъ',
 71: 'ы',
 72: 'ь',
 73: 'э',
 74: 'ю',
 75: 'я',
 76: '–',
 77: '…'}

In [11]:
vocab_size = len(chars)  # Количество уникальных символов в тексте
embedding_dim = 8  # Размерность эмбеддингов
hidden_dim = 128  # Размер скрытого слоя
sequence_length = 25  # Длина последовательности для обучения

In [12]:
# Подготовка датасета (создание входных и целевых данных для обучения)
def create_dataset(text, char_to_idx, seq_length):
    inputs = []
    targets = []
    # Формируем входные и целевые последовательности
    for i in range(0, len(text) - seq_length):
        inputs.append([char_to_idx[char] for char in text[i:i + seq_length]])
        targets.append(char_to_idx[text[i + seq_length]])
    return torch.tensor(inputs), torch.tensor(targets)

In [13]:
inputs, targets = create_dataset(text, char_to_idx, sequence_length)

In [14]:
inputs

tensor([[46,  8, 41,  ..., 36,  5,  1],
        [ 8, 41,  1,  ...,  5,  1,  2],
        [41,  1, 58,  ...,  1,  2, 54],
        ...,
        [77,  1, 46,  ..., 55, 58, 61],
        [ 1, 46, 71,  ..., 58, 61, 58],
        [46, 71,  1,  ..., 61, 58, 64]])

In [15]:
inputs.shape

torch.Size([60540, 25])

In [16]:
targets

tensor([ 2, 54, 47,  ..., 58, 64, 21])

In [17]:
targets.shape

torch.Size([60540])

In [18]:
# Определение модели RNN (модель для генерации текста)
class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(CharRNN, self).__init__()
        # Слой эмбеддингов для представления символов в виде векторов
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Рекуррентный слой (LSTM)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # Полносвязный слой для предсказания следующего символа
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x, hidden):
        # Пропускаем входные данные через слой эмбеддингов
        x = self.embedding(x)
        # Пропускаем эмбеддинги через LSTM слой
        out, hidden = self.rnn(x, hidden)
        # Применяем полносвязный слой к последнему выходу LSTM
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        # Инициализация скрытого состояния и состояния ячейки для LSTM
        return (torch.zeros(1, batch_size, hidden_dim).to(device),
                torch.zeros(1, batch_size, hidden_dim).to(device))

In [19]:
# Инициализация модели, функции потерь и оптимизатора
model = CharRNN(vocab_size, embedding_dim, hidden_dim).to(device)
loss_function = nn.CrossEntropyLoss()  # Функция потерь для классификации
optimizer = optim.Adam(model.parameters(), lr=0.003)  # Оптимизатор Adam для обновления весов

In [20]:
# Обучение модели
n_epochs = 6  # Количество эпох обучения
batch_size = 256  # Размер мини-батча

In [21]:
for epoch in range(n_epochs):
    model.train()  # Устанавливаем модель в режим обучения
    epoch_loss = 0  # Инициализация переменной для накопления потерь за эпоху
    hidden = model.init_hidden(batch_size)  # Инициализация скрытого состояния
    for i in range(0, len(inputs) - batch_size, batch_size):
        x_batch = inputs[i:i+batch_size].to(device)  # Входные данные текущего батча
        y_batch = targets[i:i+batch_size].to(device)  # Целевые данные текущего батча
        
        # Обновление скрытого состояния (чтобы избежать накопления градиентов)
        hidden = tuple([each.data for each in hidden])
        model.zero_grad()  # Обнуляем градиенты
        output, hidden = model(x_batch, hidden)  # Прямой проход через модель
        loss = loss_function(output, y_batch)  # Вычисление потерь
        loss.backward()  # Обратное распространение ошибки
        optimizer.step()  # Обновление параметров модели
        epoch_loss += loss.item()  # Накопление потерь
    
    # Печать средней потери каждые 10 эпох
    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Loss: {epoch_loss / (len(inputs) // batch_size)}')

Epoch 0, Loss: 3.0291295789055903
Epoch 1, Loss: 2.581097270472575
Epoch 2, Loss: 2.391180759769375
Epoch 3, Loss: 2.2543202441627694
Epoch 4, Loss: 2.1458877092700894
Epoch 5, Loss: 2.055717657178135


In [22]:
# Функция генерации текста (на основе обученной модели)
def generate_text(model, start_str, char_to_idx, idx_to_char, length=100):
    model.eval()  # Устанавливаем модель в режим оценки
    # Преобразование стартовой строки в тензор индексов символов
    input_chars = torch.tensor([[char_to_idx[char] for char in start_str]]).to(device)
    hidden = model.init_hidden(1)  # Инициализация скрытого состояния для генерации текста
    generated_text = start_str  # Инициализация сгенерированного текста стартовой строкой
    
    # Генерация символов по одному
    for _ in range(length):
        output, hidden = model(input_chars, hidden)  # Прямой проход через модель
        probs = torch.softmax(output, dim=1).data.cpu().numpy()  # Вычисление вероятностей для каждого символа
        char_idx = np.random.choice(len(probs[0]), p=probs[0])  # Выбор следующего символа на основе вероятностей
        generated_char = idx_to_char[char_idx]  # Преобразование индекса в символ
        generated_text += generated_char  # Добавление символа к сгенерированному тексту
        input_chars = torch.tensor([[char_idx]]).to(device)  # Обновление входных данных для следующего шага
    
    return generated_text

In [23]:
# Генерация текста
start_string = "как решал этот парадокс двадцатый век?"
generated_text = generate_text(model, start_string, char_to_idx, idx_to_char)
print("Сгенерированный текст:\n", generated_text)


Сгенерированный текст:
 как решал этот парадокс двадцатый век?
– вудыт еговний неставс и
водетет просто. вельком ом осврествииса на
че ристовделнох вылтям.
– атпо
